In [ ]:
import numpy as np
import torch
import json
import matplotlib.pyplot as plt

In [ ]:
from training import train_model, continue_training, get_list_of_models
from utils.get_device import get_device
import utils.save_load

In [ ]:
# fix random seeds
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# define path to dataset, can contain multiple files in the list
data_path = ['../data/Full-Data-80-20-split-6-kitchens-6000.json']

# get device
device = get_device()

# get list of available models to train
print(get_list_of_models())

In [ ]:
# define hyperparameters for training
NUM_EPOCHS = 600 # number of training epochs
LR = 0.0002 # learning rate
LAYERS = [512] # number of features in each layer of the network.

# train model
model, summary = train_model(
    data_paths=data_path,
    lr=LR,
    epochs=NUM_EPOCHS,
    layers=LAYERS,
    model_type='MLP_batch_norm'
    )

In [ ]:
# print the training summary
print(summary)

In [ ]:
# display the final plot
fig = summary.summary_plot
# the plot can be saved using savefig
# fig.savefig('test_figure')
fig

In [ ]:
# save model with summary
model_path = '../trained_models/example_summary'
utils.save_load.save_summary(model_path, summary)

In [ ]:
# we can continue training by passing the path to the saved summary to the continue training function 
additional_epochs = 100
retrained_model, new_summary = continue_training(model_path, data_path, additional_epochs)
print(new_summary)
new_summary.summary_plot

In [ ]:
# and we can also pass the summary directly, without saving it first
retrained_model, new_summary = continue_training(new_summary, data_path, additional_epochs)
print(new_summary)
new_summary.summary_plot

Second part: Load a trained model and use it to compute assignments.

1. load model
2. classify
```
trained_model, _ = utils.save_load.load_model_for_inference(PATH)
assignment = classify.classify(trained_model, x, 'one_hot') 
```
classify() works with lists, numpy arrays and torch tensors. Just make sure, the dimensions are correct. The most general input shape is [batch_size, num_kitchens + 1, num_items]. Specify the output format by passing for example 'one_hot' or 'label' to the classify method, see examples below. 

In [ ]:
# load trained model to perform inference
trained_model, loaded_summary = utils.save_load.load_model_for_inference(model_path)

# print training summary
print(loaded_summary)

# print model architecture
print('Model architecture:')
print(trained_model)

# show training plot
loaded_summary.summary_plot

In [ ]:
# load full data from json file and convert to numpy arrays
f = open(data_path[0])
full_data = json.load(f)
food_data = np.array(full_data['food_data_train'])  # [num_examples, num_kitchens + 1, num_items]
true_assignments = np.array(full_data['kitchens_data_train'])  # [num_examples, num_kitchens]

In [ ]:
from classify import classify
# classify full dataset and compute accuracy
pred = classify(trained_model, food_data, 'label')
accuracy = np.sum(np.where(pred - np.argmax(true_assignments, axis=1) == 0, 1, 0)) / len(food_data) * 100
print(f'Accuracy over training dataset: {accuracy :.2f}%')

In [ ]:
# generate a single test example with dimensions [num_kitchens + 1, num_items]
num_kitchens = loaded_summary.num_kitchens
num_items = loaded_summary.num_food_items

# random kitchen inventory
means = np.mean(food_data, axis=0)[:num_kitchens, :]
stds = np.std(food_data, axis=0)[:num_kitchens, :]
random_inventory = stds * np.random.randn(num_kitchens, num_items) + means

# one hot item to distribute
random_item_to_distribute = np.zeros(num_items)
random_item_to_distribute[np.random.choice(num_items)] = 100

# combine inventory and item to distribute
test_input = np.vstack((random_inventory, random_item_to_distribute))
print(f'Shape of test input: {test_input.shape}')

In [ ]:
# Compute assignments with different output encodings
assignment_default = classify(trained_model, test_input)
print(f'Assignments with probabilities: {assignment_default}')

assignment_one_hot = classify(trained_model, test_input, 'one_hot')
print(f'Assignments one hot encoded: {assignment_one_hot}')

assignment_label = classify(trained_model, test_input, 'label')
print(f'Assignments with kitchen labels: {assignment_label}')